# Imports

In [11]:
import os
import sys
import json
import pandas as pd
from itertools import combinations
import math
import statistics
from pyspark import SparkContext
import numpy as np

from pprint import pprint

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2024-03-27 12:41:09 -07:00)


# Config

In [12]:
class Path:
    curren_dir = os.getcwd()
    hw_dir = os.path.dirname(curren_dir)
    data_dir = os.path.join(hw_dir, "sample_data")

    input_csv_file = os.path.join(data_dir, "ub_sample_data.csv")

    output_dir = os.path.join(data_dir, "output")
    os.makedirs(output_dir, exist_ok=True)

    task1_output = os.path.join(output_dir, "task1_op.txt")
    task2_output = os.path.join(output_dir, "task2_op.txt")

time: 15 ms (started: 2024-03-27 12:41:11 -07:00)


# Task 1

In [14]:
import time

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, DataFrame
from graphframes import GraphFrame
from itertools import combinations
from pyspark.sql.functions import udf, col
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.3-spark3.5-s_2.13 pyspark-shell"


def read_csv(spark: SparkSession, file_path: str):
    """
    Read CSV file into DataFrame
    """
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    return df


# Define a UDF to compute combinations
@udf(returnType=ArrayType(StructType([StructField("src", StringType(), True), StructField("dst", StringType(), True)])))
def compute_combinations(user_list):
    # return list(combinations(user_list, 2))[:2]
    return user_list[0]


def construct_graph(spark: SparkSession, df: DataFrame, filter_threshold: int):
    """
    Function to construct the social network graph.
    """
    # Create user to business map
    user2bus = (
        df.groupBy("user_id")
        .agg({"business_id": "collect_set"})
        .withColumnRenamed("collect_set(business_id)", "business_set")
    )
    user2bus = user2bus.rdd.map(lambda row: (row.user_id, set(row.business_set))).collectAsMap()

    # # Filter nodes and edges
    # nodes = set()
    # edges = []

    # for pair in combinations(user2bus.keys(), 2):
    #    common_businesses = user2bus[pair[0]] & user2bus[pair[1]]
    #    if len(common_businesses)>= filter_threshold:
    #         nodes.add(pair[0])
    #         nodes.add(pair[1])
    #         edges.append((pair[0], pair[1]))
    #         edges.append((pair[1], pair[0]))
    
    # nodes = list(nodes)

    # edges_dataframe = spark.createDataFrame(edges, ["src", "dst"])


    # # Construct GraphFrame
    # # graph = GraphFrame(nodes, edges)
    # print(nodes.count())

    return user2bus


def task1(filter_threshold: int, input_file_path: str, community_output_file_path: str):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 1")
    # .set("spark.executor.memory", "4G").set("spark.driver.memory", "4G")
    sc = SparkContext(conf=conf).getOrCreate()
    spark = SparkSession(sparkContext=sc)
    spark.sparkContext.setLogLevel("ERROR")

    try:
        start_time = time.time()

        df = read_csv(spark, input_file_path)

        # Construct graph
        graph = construct_graph(spark, df, filter_threshold)
        # Plot the graph
        for k,v in graph.items():
            print(k,v)

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


# if __name__ == "__main__":
#     # Check if the correct number of command-line arguments are provided
#     if len(sys.argv) != 4:
#         print(
#             "Usage: spark-submit --packages graphframes:graphframes:0.8.2-spark3.1-s_2.12 task1.py <filter threshold>"
#             " <input file path> <community output file path>"
#         )
#         sys.exit(1)

#     # Parse command-line arguments
#     filter_threshold = int(sys.argv[1])
#     input_file_path = sys.argv[2]
#     community_output_file_path = sys.argv[3]

#     # Call task1 function
#     task1(filter_threshold, input_file_path, community_output_file_path)

task1(2, Path.input_csv_file, Path.task1_output)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 3) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apa

time: 13.5 s (started: 2024-03-27 12:42:00 -07:00)
